## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,2020-08-02 22:00:08,22.29,114.16,79.00,88,100,8.99,light rain
1,1,Mahebourg,MU,2020-08-02 22:03:47,-20.41,57.70,69.80,68,75,12.75,broken clouds
2,2,Walvis Bay,NaN,2020-08-02 21:53:55,-22.96,14.51,73.40,16,0,11.41,clear sky
3,3,Bubaque,GW,2020-08-02 22:03:47,11.28,-15.83,80.98,84,99,11.63,overcast clouds
4,4,Paragominas,BR,2020-08-02 22:01:07,-2.97,-47.48,82.40,56,76,2.82,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,2020-08-02 22:00:08,22.29,114.16,79.00,88,100,8.99,light rain
3,3,Bubaque,GW,2020-08-02 22:03:47,11.28,-15.83,80.98,84,99,11.63,overcast clouds
4,4,Paragominas,BR,2020-08-02 22:01:07,-2.97,-47.48,82.40,56,76,2.82,broken clouds
7,7,Georgetown,MY,2020-08-02 22:01:45,5.41,100.34,82.40,78,20,5.82,few clouds
18,18,Yellowknife,CA,2020-08-02 22:00:35,62.46,-114.35,80.60,54,75,17.22,broken clouds
21,21,Hilo,US,2020-08-02 22:03:50,19.73,-155.09,82.40,58,90,6.93,overcast clouds
23,23,Luang Prabang,LA,2020-08-02 21:54:00,19.89,102.13,75.99,89,100,2.73,light rain
29,29,Quisqueya,DO,2020-08-02 22:03:51,18.56,-69.41,87.80,70,40,6.93,scattered clouds
30,30,Bethel,US,2020-08-02 22:03:51,41.37,-73.41,84.20,79,40,11.41,scattered clouds
31,31,Acapulco,MX,2020-08-02 22:02:00,16.86,-99.89,89.60,70,20,6.93,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Date                   229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Date                   229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,79.00,light rain,22.29,114.16,NaN
3,Bubaque,GW,80.98,overcast clouds,11.28,-15.83,NaN
4,Paragominas,BR,82.40,broken clouds,-2.97,-47.48,NaN
7,Georgetown,MY,82.40,few clouds,5.41,100.34,NaN
18,Yellowknife,CA,80.60,broken clouds,62.46,-114.35,NaN
21,Hilo,US,82.40,overcast clouds,19.73,-155.09,NaN
23,Luang Prabang,LA,75.99,light rain,19.89,102.13,NaN
29,Quisqueya,DO,87.80,scattered clouds,18.56,-69.41,NaN
30,Bethel,US,84.20,scattered clouds,41.37,-73.41,NaN
31,Acapulco,MX,89.60,few clouds,16.86,-99.89,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,79.00,light rain,22.29,114.16,Mini Hotel Central
3,Bubaque,GW,80.98,overcast clouds,11.28,-15.83,Casa Dora
4,Paragominas,BR,82.40,broken clouds,-2.97,-47.48,HOTEL RODA VIVA
7,Georgetown,MY,82.40,few clouds,5.41,100.34,Cititel Penang
18,Yellowknife,CA,80.60,broken clouds,62.46,-114.35,The Explorer Hotel


In [10]:
hotel_df.count()

City                   229
Country                229
Max Temp               229
Current Description    229
Lat                    229
Lng                    229
Hotel Name             216
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   216
Country                216
Max Temp               216
Current Description    216
Lat                    216
Lng                    216
Hotel Name             216
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))